In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import hvplot.pandas
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="ChrisClass")
from scipy.stats import linregress
from citipy import citipy

In [2]:
#Upload landfall csv to notebook
landfall_df = pd.read_csv("Resources/landfall.csv")
#Upload costliest hurricane csv to notebook
damage_df = pd.read_csv("Resources/costliness.csv")

#Merge the two data frames into one
merged_df = pd.merge(landfall_df,damage_df,on = ['Season','Name'],how='outer')

#Drop values with no location data
merged_df = merged_df.dropna(subset=['Latitude'])

#Rename Season columns to Year
merged_df = merged_df.rename(columns={'Season':'Year'})

#Show dataframe preview
merged_df.head()


,#,Date,Year,Time,Latitude,Longitude,Max Winds (kt),SS HWS,RMW nm,Central Prssure (mb),States Affected,Name,Category,Adjusted Costs
1,3.0,8/18/83,1983,0700Z,29.1,-95.1,100.0,3.0,10,962,CTX3,Alicia,3.0,$9.4B
2,10.0,9/13/84,1984,0700Z,33.9,-78.0,95.0,2.0,15,979,NC2,Diana,NaN,NaN
3,2.0,7/25/85,1985,0300Z,32.2,-80.5,65.0,1.0,20,1003,SC1,Bob,NaN,NaN
4,4.0,8/15/85,1985,1630Z,29.6,-92.7,80.0,1.0,15,987,LA1,Danny,NaN,NaN
5,5.0,9/2/85,1985,1300Z,30.4,-89.2,100.0,3.0,10,959,"AL3,MS3,AFL3",Elena,3.0,$3.8B


In [3]:
#Create lists to store states, counties, and coordinates.
counties = []
states = []
lat_lngs = []

#Convert Lat and Long to a string
merged_df['Longitude'] = merged_df['Longitude'].astype(str)
merged_df['Latitude'] = merged_df['Latitude'].astype(str)

#Add lattitude and longitude values to list
lat_lngs = merged_df[['Latitude','Longitude']].values.tolist()

#Loop through lat_lng list to pull county and state names with geopy  
for lat_lng in lat_lngs:
    try:
        county = geolocator.reverse(lat_lng[0]+","+lat_lng[1])
        county = county.raw['address']['county']
        counties.append(county)
        state = geolocator.reverse(lat_lng[0]+","+lat_lng[1])
        state = state.raw['address']['state']
        states.append(state)
    #To handle errors if data isn't found
    except:
        counties.append('')
        states.append('')

#Add county and state names to dataframe
merged_df['County'] = counties
merged_df['State'] = states

merged_df.head()

,#,Date,Year,Time,Latitude,Longitude,Max Winds (kt),SS HWS,RMW nm,Central Prssure (mb),States Affected,Name,Category,Adjusted Costs,County,State
1,3.0,8/18/83,1983,0700Z,29.1,-95.1,100.0,3.0,10,962,CTX3,Alicia,3.0,$9.4B,Galveston County,Texas
2,10.0,9/13/84,1984,0700Z,33.9,-78.0,95.0,2.0,15,979,NC2,Diana,NaN,NaN,Brunswick County,North Carolina
3,2.0,7/25/85,1985,0300Z,32.2,-80.5,65.0,1.0,20,1003,SC1,Bob,NaN,NaN,,
4,4.0,8/15/85,1985,1630Z,29.6,-92.7,80.0,1.0,15,987,LA1,Danny,NaN,NaN,Cameron Parish,Louisiana
5,5.0,9/2/85,1985,1300Z,30.4,-89.2,100.0,3.0,10,959,"AL3,MS3,AFL3",Elena,3.0,$3.8B,Harrison County,Mississippi


In [4]:
#Save new csv to avoid long run time of above code 
merged_df.to_csv('Resources/merged_df.csv',index=False)

In [3]:
#Upload new merged csv to notebook
hurricanes_df = pd.read_csv('Resources/merged_df.csv')
hurricanes_df

,#,Date,Year,Time,Latitude,Longitude,Max Winds (kt),SS HWS,RMW nm,Central Prssure (mb),States Affected,Name,Category,Adjusted Costs,County,State
0,3.0,8/18/83,1983,0700Z,29.1,-95.1,100.0,3.0,10,962,CTX3,Alicia,3.0,$9.4B,Galveston County,Texas
1,10.0,9/13/84,1984,0700Z,33.9,-78.0,95.0,2.0,15,979,NC2,Diana,NaN,NaN,Brunswick County,North Carolina
2,2.0,7/25/85,1985,0300Z,32.2,-80.5,65.0,1.0,20,1003,SC1,Bob,NaN,NaN,NaN,NaN
3,4.0,8/15/85,1985,1630Z,29.6,-92.7,80.0,1.0,15,987,LA1,Danny,NaN,NaN,Cameron Parish,Louisiana
4,5.0,9/2/85,1985,1300Z,30.4,-89.2,100.0,3.0,10,959,"AL3,MS3,AFL3",Elena,3.0,$3.8B,Harrison County,Mississippi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,9.0,9/28/22,2022,0200Z,24.6,-82.9,110.0,3.0,20,952,BFL3,Ian,4.0,$118.5B,Monroe County,Florida
92,9.0,9/28/22,2022,1905Z,26.7,-82.2,130.0,4.0,20,941,BFL4,Ian,4.0,$118.5B,Lee County,Florida
93,9.0,9/28/22,2022,2035Z,26.8,-82.0,125.0,4.0,20,945,"BFL4,ICFL1,DFL1",Ian,4.0,$118.5B,Charlotte County,Florida
94,9.0,9/30/22,2022,1805Z,33.3,-79.2,70.0,1.0,40,978,SC1,Ian,4.0,$118.5B,Georgetown County,South Carolina


In [4]:
#Drop rows with empty values
hurricanes_df.dropna(axis=0, how='any', inplace=True)
hurricanes_df.head()

,#,Date,Year,Time,Latitude,Longitude,Max Winds (kt),SS HWS,RMW nm,Central Prssure (mb),States Affected,Name,Category,Adjusted Costs,County,State
0,3.0,8/18/83,1983,0700Z,29.1,-95.1,100.0,3.0,10,962,CTX3,Alicia,3.0,$9.4B,Galveston County,Texas
4,5.0,9/2/85,1985,1300Z,30.4,-89.2,100.0,3.0,10,959,"AL3,MS3,AFL3",Elena,3.0,$3.8B,Harrison County,Mississippi
7,12.0,10/29/85,1985,1100Z,29.5,-91.4,75.0,1.0,15,971,LA1,Juan,1.0,$4.3B,St. Mary Parish,Louisiana
14,11.0,9/22/89,1989,0400Z,32.8,-79.8,120.0,4.0,20,934,"SC4,INC1",Hugo,4.0,$22.7B,Charleston County,South Carolina
16,3.0,8/19/91,1991,1720Z,41.2,-71.6,90.0,2.0,25,962,"RI2,MA2,NY2,CT2",Bob,2.0,$3.5B,South County,Rhode Island


In [5]:
#Dictionary to store State names and abb.
state_to_abbrev = {"Alabama": "AL","Alaska": "AK","Arizona": "AZ","Arkansas": "AR","California": "CA",
    "Colorado": "CO","Connecticut": "CT","Delaware": "DE","Florida": "FL","Georgia": "GA",
    "Hawaii": "HI","Idaho": "ID","Illinois": "IL","Indiana": "IN","Iowa": "IA",
    "Kansas": "KS","Kentucky": "KY","Louisiana": "LA","Maine": "ME","Maryland": "MD",
    "Massachusetts": "MA","Michigan": "MI","Minnesota": "MN","Mississippi": "MS","Missouri": "MO",
    "Montana": "MT","Nebraska": "NE","Nevada": "NV","New Hampshire": "NH","New Jersey": "NJ",
    "New Mexico": "NM","New York": "NY","North Carolina": "NC","North Dakota": "ND","Ohio": "OH",
    "Oklahoma": "OK","Oregon": "OR","Pennsylvania": "PA","Rhode Island": "RI","South Carolina": "SC",
    "South Dakota": "SD","Tennessee": "TN","Texas": "TX","Utah": "UT","Vermont": "VT",
    "Virginia": "VA","Washington": "WA","West Virginia": "WV","Wisconsin": "WI","Wyoming": "WY","District of Columbia": "DC"}

#Replace State Names with codes
hurricanes_df['State'] = hurricanes_df['State'].replace(state_to_abbrev)
hurricanes_df.head()

,#,Date,Year,Time,Latitude,Longitude,Max Winds (kt),SS HWS,RMW nm,Central Prssure (mb),States Affected,Name,Category,Adjusted Costs,County,State
0,3.0,8/18/83,1983,0700Z,29.1,-95.1,100.0,3.0,10,962,CTX3,Alicia,3.0,$9.4B,Galveston County,TX
4,5.0,9/2/85,1985,1300Z,30.4,-89.2,100.0,3.0,10,959,"AL3,MS3,AFL3",Elena,3.0,$3.8B,Harrison County,MS
7,12.0,10/29/85,1985,1100Z,29.5,-91.4,75.0,1.0,15,971,LA1,Juan,1.0,$4.3B,St. Mary Parish,LA
14,11.0,9/22/89,1989,0400Z,32.8,-79.8,120.0,4.0,20,934,"SC4,INC1",Hugo,4.0,$22.7B,Charleston County,SC
16,3.0,8/19/91,1991,1720Z,41.2,-71.6,90.0,2.0,25,962,"RI2,MA2,NY2,CT2",Bob,2.0,$3.5B,South County,RI


In [6]:
#Make a list to store cities and coordinates 
cities= []
lat_lngs2= []

#Convert Lat and Long to a float
hurricanes_df['Longitude'] = hurricanes_df['Longitude'].astype(float)
hurricanes_df['Latitude'] = hurricanes_df['Latitude'].astype(float)

#Add lattitude and longitude values to list
lat_lngs2 = hurricanes_df[['Latitude','Longitude']].values.tolist()

#Loop through lat_lng list to pull cities names with citipy  
for lat_lng2 in lat_lngs2:
    city = citipy.nearest_city(lat_lng2[0], lat_lng2[1]).city_name
    cities.append(city)

#Add city names to dataframe
hurricanes_df['City'] =cities
hurricanes_df.head()

,#,Date,Year,Time,Latitude,Longitude,Max Winds (kt),SS HWS,RMW nm,Central Prssure (mb),States Affected,Name,Category,Adjusted Costs,County,State,City
0,3.0,8/18/83,1983,0700Z,29.1,-95.1,100.0,3.0,10,962,CTX3,Alicia,3.0,$9.4B,Galveston County,TX,hitchcock
4,5.0,9/2/85,1985,1300Z,30.4,-89.2,100.0,3.0,10,959,"AL3,MS3,AFL3",Elena,3.0,$3.8B,Harrison County,MS,long beach
7,12.0,10/29/85,1985,1100Z,29.5,-91.4,75.0,1.0,15,971,LA1,Juan,1.0,$4.3B,St. Mary Parish,LA,patterson
14,11.0,9/22/89,1989,0400Z,32.8,-79.8,120.0,4.0,20,934,"SC4,INC1",Hugo,4.0,$22.7B,Charleston County,SC,mount pleasant
16,3.0,8/19/91,1991,1720Z,41.2,-71.6,90.0,2.0,25,962,"RI2,MA2,NY2,CT2",Bob,2.0,$3.5B,South County,RI,charlestown


In [7]:
#Remove "County" amd "Parish" from county column
hurricanes_df['County'] = hurricanes_df['County'].str.replace(' County','')
hurricanes_df['County'] = hurricanes_df['County'].str.replace(' Parish','')
hurricanes_df.head()

,#,Date,Year,Time,Latitude,Longitude,Max Winds (kt),SS HWS,RMW nm,Central Prssure (mb),States Affected,Name,Category,Adjusted Costs,County,State,City
0,3.0,8/18/83,1983,0700Z,29.1,-95.1,100.0,3.0,10,962,CTX3,Alicia,3.0,$9.4B,Galveston,TX,hitchcock
4,5.0,9/2/85,1985,1300Z,30.4,-89.2,100.0,3.0,10,959,"AL3,MS3,AFL3",Elena,3.0,$3.8B,Harrison,MS,long beach
7,12.0,10/29/85,1985,1100Z,29.5,-91.4,75.0,1.0,15,971,LA1,Juan,1.0,$4.3B,St. Mary,LA,patterson
14,11.0,9/22/89,1989,0400Z,32.8,-79.8,120.0,4.0,20,934,"SC4,INC1",Hugo,4.0,$22.7B,Charleston,SC,mount pleasant
16,3.0,8/19/91,1991,1720Z,41.2,-71.6,90.0,2.0,25,962,"RI2,MA2,NY2,CT2",Bob,2.0,$3.5B,South,RI,charlestown


In [8]:
#Upload HPI csv to dataframe
hpi_df = pd.read_csv('Resources/hpi_county.csv')
hpi_df.replace('.', "", inplace=True)
hpi_df.head()

,State,County,FIPS code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base
0,AL,Autauga,1001,1986,,100.00,95.59,70.96
1,AL,Autauga,1001,1987,-1.94,98.06,93.74,69.59
2,AL,Autauga,1001,1988,2.57,100.58,96.14,71.38
3,AL,Autauga,1001,1989,4.32,104.92,100.29,74.46
4,AL,Autauga,1001,1990,-0.29,104.62,100.00,74.24


In [9]:
#Create lists to store the year values of the years around a hurricane strike
previous_year = []
year_1 = []
year_2 = []
year_3 = []

#Loop through the hpi data and pull the previous year value
for index in hpi_df.index:
    try:
        value = hpi_df.at[index-1, 'Year']
        previous_year.append(value)
    except:
        previous_year.append('')

#Loop through the hpi data and pull the next year value
for index in hpi_df.index:
    try:
        value = hpi_df.at[index+1, 'Year']
        year_1.append(value)
    except:
        year_1.append('')

#Loop through the hpi data and pull two year values ahead
for index in hpi_df.index:
    try:
        value = hpi_df.at[index+2, 'Year']
        year_2.append(value)
    except:
        year_2.append('')

#Loop through the hpi data and pull three year values ahead
for index in hpi_df.index:
    try:
        value = hpi_df.at[index+3, 'Year']
        year_3.append(value)
    except:
        year_3.append('')

#Create new columns and place the values on the data frame
hpi_df['Year -1'] = previous_year
hpi_df['Year +1'] = year_1
hpi_df['Year +2'] = year_2
hpi_df['Year +3'] = year_3
hpi_df.head()


,State,County,FIPS code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base,Year -1,Year +1,Year +2,Year +3
0,AL,Autauga,1001,1986,,100.00,95.59,70.96,,1987,1988,1989
1,AL,Autauga,1001,1987,-1.94,98.06,93.74,69.59,1986,1988,1989,1990
2,AL,Autauga,1001,1988,2.57,100.58,96.14,71.38,1987,1989,1990,1991
3,AL,Autauga,1001,1989,4.32,104.92,100.29,74.46,1988,1990,1991,1992
4,AL,Autauga,1001,1990,-0.29,104.62,100.00,74.24,1989,1991,1992,1993


In [10]:
#Create lists to store the Annual change values of the years around a hurricane strike
prev_ac = []
year_1_ac = []
year_2_ac = []
year_3_ac = []

#Loop through the hpi data and pull the previous ac value
for index in hpi_df.index:
    try:
        value = hpi_df.at[index-1, 'Annual Change (%)']
        prev_ac.append(value)
    except:
        prev_ac.append('')

#Loop through the hpi data and pull the next ac value
for index in hpi_df.index:
    try:
        value = hpi_df.at[index+1, 'Annual Change (%)']
        year_1_ac.append(value)
    except:
        year_1_ac.append('')

#Loop through the hpi data and pull two ac values ahead
for index in hpi_df.index:
    try:
        value = hpi_df.at[index+2, 'Annual Change (%)']
        year_2_ac.append(value)
    except:
        year_2_ac.append('')

#Loop through the hpi data and pull three ac values ahead
for index in hpi_df.index:
    try:
        value = hpi_df.at[index+3, 'Annual Change (%)']
        year_3_ac.append(value)
    except:
        year_3_ac.append('')

#Create new columns and place the values on the data frame
hpi_df['Annual Change (%) -1'] = prev_ac
hpi_df['Annual Change (%) +1'] = year_1_ac
hpi_df['Annual Change (%) +2'] = year_2_ac
hpi_df['Annual Change (%) +3'] = year_3_ac
hpi_df.head()

,State,County,FIPS code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base,Year -1,Year +1,Year +2,Year +3,Annual Change (%) -1,Annual Change (%) +1,Annual Change (%) +2,Annual Change (%) +3
0,AL,Autauga,1001,1986,,100.00,95.59,70.96,,1987,1988,1989,,-1.94,2.57,4.32
1,AL,Autauga,1001,1987,-1.94,98.06,93.74,69.59,1986,1988,1989,1990,,2.57,4.32,-0.29
2,AL,Autauga,1001,1988,2.57,100.58,96.14,71.38,1987,1989,1990,1991,-1.94,4.32,-0.29,2.52
3,AL,Autauga,1001,1989,4.32,104.92,100.29,74.46,1988,1990,1991,1992,2.57,-0.29,2.52,3.09
4,AL,Autauga,1001,1990,-0.29,104.62,100.00,74.24,1989,1991,1992,1993,4.32,2.52,3.09,3.35


In [11]:
#Create lists to store the HPI values of the years around a hurricane strike
prev_hpi = []
year_1_hpi = []
year_2_hpi = []
year_3_hpi = []

#Loop through the hpi data and pull the previous HPI value
for index in hpi_df.index:
    try:
        value = hpi_df.at[index-1, 'HPI']
        prev_hpi.append(value)
    except:
        prev_hpi.append('')

#Loop through the hpi data and pull the next HPI value
for index in hpi_df.index:
    try:
        value = hpi_df.at[index+1, 'HPI']
        year_1_hpi.append(value)
    except:
        year_1_hpi.append('')

#Loop through the hpi data and pull two HPI value ahead
for index in hpi_df.index:
    try:
        value = hpi_df.at[index+2, 'HPI']
        year_2_hpi.append(value)
    except:
        year_2_hpi.append('')

#Loop through the hpi data and pull three HPI value ahead
for index in hpi_df.index:
    try:
        value = hpi_df.at[index+3, 'HPI']
        year_3_hpi.append(value)
    except:
        year_3_hpi.append('')

#Create new columns and place the values on the data frame
hpi_df['HPI -1'] = prev_hpi
hpi_df['HPI +1'] = year_1_hpi
hpi_df['HPI +2'] = year_2_hpi
hpi_df['HPI +3'] = year_3_hpi
hpi_df.head()

,State,County,FIPS code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base,Year -1,Year +1,Year +2,Year +3,Annual Change (%) -1,Annual Change (%) +1,Annual Change (%) +2,Annual Change (%) +3,HPI -1,HPI +1,HPI +2,HPI +3
0,AL,Autauga,1001,1986,,100.00,95.59,70.96,,1987,1988,1989,,-1.94,2.57,4.32,,98.06,100.58,104.92
1,AL,Autauga,1001,1987,-1.94,98.06,93.74,69.59,1986,1988,1989,1990,,2.57,4.32,-0.29,100.00,100.58,104.92,104.62
2,AL,Autauga,1001,1988,2.57,100.58,96.14,71.38,1987,1989,1990,1991,-1.94,4.32,-0.29,2.52,98.06,104.92,104.62,107.25
3,AL,Autauga,1001,1989,4.32,104.92,100.29,74.46,1988,1990,1991,1992,2.57,-0.29,2.52,3.09,100.58,104.62,107.25,110.57
4,AL,Autauga,1001,1990,-0.29,104.62,100.00,74.24,1989,1991,1992,1993,4.32,2.52,3.09,3.35,104.92,107.25,110.57,114.28


In [12]:
#Merge HPI data with hurricane data
hurricane_hpi_df = pd.merge(hurricanes_df,hpi_df,on=['Year','County','State'],how='inner')
hurricane_hpi_df.head()

,#,Date,Year,Time,Latitude,Longitude,Max Winds (kt),SS HWS,RMW nm,Central Prssure (mb),...,Year +2,Year +3,Annual Change (%) -1,Annual Change (%) +1,Annual Change (%) +2,Annual Change (%) +3,HPI -1,HPI +1,HPI +2,HPI +3
0,3.0,8/18/83,1983,0700Z,29.1,-95.1,100.0,3.0,10,962,...,1985,1986,-2.03,1.64,-4.40,-1.16,161.13,167.34,159.99,158.13
1,5.0,9/2/85,1985,1300Z,30.4,-89.2,100.0,3.0,10,959,...,1987,1988,4.57,3.03,-5.08,-0.86,177.04,196.94,186.93,185.32
2,11.0,9/22/89,1989,0400Z,32.8,-79.8,120.0,4.0,20,934,...,1991,1992,5.89,4.00,2.84,2.40,263.90,285.80,293.90,300.97
3,4.0,8/24/92,1992,0905Z,25.5,-80.3,145.0,5.0,10,922,...,1994,1995,2.09,6.14,4.36,4.63,239.18,261.13,272.52,285.14
4,17.0,10/4/95,1995,2200Z,30.3,-87.1,100.0,3.0,50,942,...,1997,1998,3.43,5.35,5.62,3.30,203.44,224.56,237.17,245.00


In [13]:
#Make a new dataframe with only relevant columns
clean_hurricane_hpi_df = hurricane_hpi_df[['Name','Max Winds (kt)','Adjusted Costs','City','County','State',
                        'Year -1','Year','Year +1','Year +2',
                        'Annual Change (%) -1','Annual Change (%)','Annual Change (%) +1','Annual Change (%) +2',
                        'HPI -1','HPI','HPI +1','HPI +2']]
clean_hurricane_hpi_df = clean_hurricane_hpi_df[(clean_hurricane_hpi_df['Year'] < 2022) & (clean_hurricane_hpi_df['Year'] > 1980)]
clean_hurricane_hpi_df.head()

,Name,Max Winds (kt),Adjusted Costs,City,County,State,Year -1,Year,Year +1,Year +2,Annual Change (%) -1,Annual Change (%),Annual Change (%) +1,Annual Change (%) +2,HPI -1,HPI,HPI +1,HPI +2
0,Alicia,100.0,$9.4B,hitchcock,Galveston,TX,1982,1983,1984,1985,-2.03,2.17,1.64,-4.40,161.13,164.63,167.34,159.99
1,Elena,100.0,$3.8B,long beach,Harrison,MS,1984,1985,1986,1987,4.57,7.97,3.03,-5.08,177.04,191.15,196.94,186.93
2,Hugo,120.0,$22.7B,mount pleasant,Charleston,SC,1988,1989,1990,1991,5.89,4.13,4.00,2.84,263.90,274.79,285.80,293.90
3,Andrew,145.0,$60.2B,lakes by the bay,Miami-Dade,FL,1991,1992,1993,1994,2.09,2.86,6.14,4.36,239.18,246.02,261.13,272.52
4,Opal,100.0,$9.6B,gulf breeze,Escambia,FL,1994,1995,1996,1997,3.43,4.78,5.35,5.62,203.44,213.16,224.56,237.17


In [14]:
#Save hurricanes df to directory
hurricanes_df.to_csv('Output_CSVs/hurricanes_df.csv', index=False)

#Save Merged df to directory
hpi_df.to_csv('Output_CSVs/hpi_df.csv', index=False)

#Save clean df as CSV to directory
clean_hurricane_hpi_df.to_csv('Output_CSVs/clean_hurricane_hpi_data.csv', index=False)



hurricanes_df = Total Data for hurricane strikes in the US

hpi_df = Total HPI data with new columns for every county

clean_hurricane_hpi_df = Comined previous two data frames. All hurricane strikes with HPI data.

In [15]:
hurricanes_df

,#,Date,Year,Time,Latitude,Longitude,Max Winds (kt),SS HWS,RMW nm,Central Prssure (mb),States Affected,Name,Category,Adjusted Costs,County,State,City
0,3.0,8/18/83,1983,0700Z,29.1,-95.1,100.0,3.0,10,962,CTX3,Alicia,3.0,$9.4B,Galveston,TX,hitchcock
4,5.0,9/2/85,1985,1300Z,30.4,-89.2,100.0,3.0,10,959,"AL3,MS3,AFL3",Elena,3.0,$3.8B,Harrison,MS,long beach
7,12.0,10/29/85,1985,1100Z,29.5,-91.4,75.0,1.0,15,971,LA1,Juan,1.0,$4.3B,St. Mary,LA,patterson
14,11.0,9/22/89,1989,0400Z,32.8,-79.8,120.0,4.0,20,934,"SC4,INC1",Hugo,4.0,$22.7B,Charleston,SC,mount pleasant
16,3.0,8/19/91,1991,1720Z,41.2,-71.6,90.0,2.0,25,962,"RI2,MA2,NY2,CT2",Bob,2.0,$3.5B,South,RI,charlestown
17,4.0,8/24/92,1992,0905Z,25.5,-80.3,145.0,5.0,10,922,"CFL5,BFL4",Andrew,5.0,$60.2B,Miami-Dade,FL,lakes by the bay
18,4.0,8/26/92,1992,0830Z,29.6,-91.5,100.0,3.0,20,956,LA3,Andrew,5.0,$60.2B,St. Mary,LA,franklin
23,17.0,10/4/95,1995,2200Z,30.3,-87.1,100.0,3.0,50,942,"AFL3,IAL1",Opal,3.0,$9.6B,Escambia,FL,gulf breeze
25,6.0,9/6/96,1996,0030Z,33.9,-78.0,100.0,3.0,40,954,NC3,Fran,3.0,$10.0B,Brunswick,NC,long beach
31,7.0,9/25/98,1998,1530Z,24.5,-81.8,90.0,2.0,25,981,BFL2,Georges,2.0,$11.5B,Monroe,FL,key west


In [16]:
hpi_df

,State,County,FIPS code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base,Year -1,Year +1,Year +2,Year +3,Annual Change (%) -1,Annual Change (%) +1,Annual Change (%) +2,Annual Change (%) +3,HPI -1,HPI +1,HPI +2,HPI +3
0,AL,Autauga,1001,1986,,100.00,95.59,70.96,,1987,1988,1989,,-1.94,2.57,4.32,,98.06,100.58,104.92
1,AL,Autauga,1001,1987,-1.94,98.06,93.74,69.59,1986,1988,1989,1990,,2.57,4.32,-0.29,100.00,100.58,104.92,104.62
2,AL,Autauga,1001,1988,2.57,100.58,96.14,71.38,1987,1989,1990,1991,-1.94,4.32,-0.29,2.52,98.06,104.92,104.62,107.25
3,AL,Autauga,1001,1989,4.32,104.92,100.29,74.46,1988,1990,1991,1992,2.57,-0.29,2.52,3.09,100.58,104.62,107.25,110.57
4,AL,Autauga,1001,1990,-0.29,104.62,100.00,74.24,1989,1991,1992,1993,4.32,2.52,3.09,3.35,104.92,107.25,110.57,114.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100427,WY,Weston,56045,2019,8.33,223.40,,201.09,2018,2020,2021,2022,-0.90,4.36,4.93,6.87,206.21,233.13,244.62,261.41
100428,WY,Weston,56045,2020,4.36,233.13,,209.86,2019,2021,2022,2023,8.33,4.93,6.87,-1.17,223.40,244.62,261.41,258.37
100429,WY,Weston,56045,2021,4.93,244.62,,220.20,2020,2022,2023,,4.36,6.87,-1.17,,233.13,261.41,258.37,
100430,WY,Weston,56045,2022,6.87,261.41,,235.32,2021,2023,,,4.93,-1.17,,,244.62,258.37,,


In [17]:
clean_hurricane_hpi_df

,Name,Max Winds (kt),Adjusted Costs,City,County,State,Year -1,Year,Year +1,Year +2,Annual Change (%) -1,Annual Change (%),Annual Change (%) +1,Annual Change (%) +2,HPI -1,HPI,HPI +1,HPI +2
0,Alicia,100.0,$9.4B,hitchcock,Galveston,TX,1982,1983,1984,1985,-2.03,2.17,1.64,-4.40,161.13,164.63,167.34,159.99
1,Elena,100.0,$3.8B,long beach,Harrison,MS,1984,1985,1986,1987,4.57,7.97,3.03,-5.08,177.04,191.15,196.94,186.93
2,Hugo,120.0,$22.7B,mount pleasant,Charleston,SC,1988,1989,1990,1991,5.89,4.13,4.00,2.84,263.90,274.79,285.80,293.90
3,Andrew,145.0,$60.2B,lakes by the bay,Miami-Dade,FL,1991,1992,1993,1994,2.09,2.86,6.14,4.36,239.18,246.02,261.13,272.52
4,Opal,100.0,$9.6B,gulf breeze,Escambia,FL,1994,1995,1996,1997,3.43,4.78,5.35,5.62,203.44,213.16,224.56,237.17
5,Fran,100.0,$10.0B,long beach,Brunswick,NC,1995,1996,1997,1998,8.11,5.59,8.86,7.20,192.77,203.54,221.57,237.51
6,Georges,90.0,$11.5B,key west,Monroe,FL,1997,1998,1999,2000,6.20,7.37,5.86,14.89,292.04,313.55,331.92,381.35
7,Georges,90.0,$11.5B,biloxi,Harrison,MS,1997,1998,1999,2000,4.08,5.31,5.74,5.58,243.21,256.11,270.82,285.95
8,Floyd,90.0,$12.1B,long beach,Brunswick,NC,1998,1999,2000,2001,7.20,6.14,8.12,5.06,237.51,252.09,272.55,286.34
9,Isabel,90.0,$9.3B,morehead city,Carteret,NC,2002,2003,2004,2005,3.61,4.73,8.95,22.29,205.27,214.98,234.23,286.44
